In [5]:
from dotenv import load_dotenv
import pandas as pd
import numpy as np
import pymongo
import os

In [6]:
load_dotenv()

True

In [7]:
client = pymongo.MongoClient(os.getenv('MONGO_URI'))
db = client[os.getenv('MONGO_DATABASE')]
collection = db['scrapy_colombia_apartments']

df = pd.DataFrame(list(collection.find()))
df = df.drop(columns=['_id'], axis=1)

df.head(3)

,codigo,ciudad,tipo_propiedad,tipo_operacion,precio_venta,area,habitaciones,banos,administracion,parqueaderos,...,website,last_view,datetime,precio_arriendo,longitud,latitud,fecha_actualizacion_precio_arriendo,precio_arriendo_anterior,fecha_actualizacion_precio_venta,precio_venta_anterior
0,4933-M3452382,CHIA,APARTAMENTO,VENTA,225000000.0,56.03,2.0,2.0,120000.0,0.0,...,metrocuadrado.com,2023-09-23 19:26:48.110,2023-09-22 19:32:52.583,NaN,NaN,NaN,NaT,NaN,NaT,NaN
1,3165-M4202228,CHIA,APARTAMENTO,VENTA,290000000.0,83.00,2.0,2.0,NaN,2.0,...,metrocuadrado.com,2023-09-23 19:50:14.971,2023-09-22 19:32:55.890,NaN,NaN,NaN,NaT,NaN,NaT,NaN
2,13436-M4550058,CHIA,APARTAMENTO,VENTA,215000000.0,44.00,1.0,2.0,242400.0,1.0,...,metrocuadrado.com,2023-09-23 19:45:50.662,2023-09-22 19:32:58.950,NaN,NaN,NaN,NaT,NaN,NaT,NaN


In [8]:
images_explode = df.explode('imagenes')
images_explode = images_explode.dropna(subset=['imagenes'])

In [9]:
images_df = images_explode[['codigo', 'imagenes']].rename(columns={'imagenes': 'url_imagen'})
images_df.shape

(298575, 2)

In [10]:
images_df.to_csv('../data/processed/images.csv', index=False)
images_df.columns

Index(['codigo', 'url_imagen'], dtype='object')

In [11]:
from src import extract_features

In [12]:
df['jacuzzi'] = df.featured_interior.apply(extract_features.check_jacuzzi)
df['piso'] = df.featured_interior.apply(extract_features.extract_piso)
df['closets'] = df.featured_interior.apply(extract_features.extract_closets)
df['chimenea'] = df.featured_interior.apply(extract_features.check_chimeny)
df['permite_mascotas'] = df.featured_interior.apply(extract_features.check_mascotas)
df['gimnasio'] = df.featured_zona_comun.apply(extract_features.check_gimnasio)
df['ascensor'] = df.featured_exterior.apply(extract_features.check_ascensor)
df['conjunto_cerrado'] = df.featured_exterior.apply(extract_features.check_conjunto_cerrado)

In [13]:
df = df.drop(columns=['featured_interior', 'featured_zona_comun', 'featured_exterior', 'featured_sector', 'imagenes'], axis=1)
df.columns

Index(['codigo', 'ciudad', 'tipo_propiedad', 'tipo_operacion', 'precio_venta',
       'area', 'habitaciones', 'banos', 'administracion', 'parqueaderos',
       'sector', 'sector_comun', 'estrato', 'antiguedad', 'estado', 'compañia',
       'descripcion', 'website', 'last_view', 'datetime', 'precio_arriendo',
       'longitud', 'latitud', 'fecha_actualizacion_precio_arriendo',
       'precio_arriendo_anterior', 'fecha_actualizacion_precio_venta',
       'precio_venta_anterior', 'jacuzzi', 'piso', 'closets', 'chimenea',
       'permite_mascotas', 'gimnasio', 'ascensor', 'conjunto_cerrado'],
      dtype='object')

In [15]:
df.to_csv('../data/interim/apartments.csv', index=False)

In [16]:
import session_info

session_info.show()